<a href="https://colab.research.google.com/github/anphantt2406/Heritage-Health-Prize/blob/main/PrepareData_DroppedVer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/HHP_release3/

Mounted at /content/drive
/content/drive/My Drive/HHP_release3


# Import Library

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()

# Member process

In [3]:
member_df = pd.read_csv("Members.csv")
#member_df.info()

### Gender - Onehot Encoding

In [4]:
member_df['Male'] = 0
member_df['Female'] = 0
member_df['Unknown'] = 0
member_df.loc[member_df.Sex == 'M', 'Male'] = 1
member_df.loc[member_df.Sex == 'F', 'Female'] = 1
member_df.loc[member_df.Sex.isna(), 'Unknown'] = 1
member_df.drop(columns=['Sex'], inplace=True)

### Age - Replace by Mean

In [5]:
member_df.loc[member_df.AgeAtFirstClaim == '0-9', 'AgeAtFirstClaim'] = 5
member_df.loc[member_df.AgeAtFirstClaim == '10-19', 'AgeAtFirstClaim'] = 15
member_df.loc[member_df.AgeAtFirstClaim == '20-29', 'AgeAtFirstClaim'] = 25
member_df.loc[member_df.AgeAtFirstClaim == '30-39', 'AgeAtFirstClaim'] = 35
member_df.loc[member_df.AgeAtFirstClaim == '40-49', 'AgeAtFirstClaim'] = 45
member_df.loc[member_df.AgeAtFirstClaim == '50-59', 'AgeAtFirstClaim'] = 55
member_df.loc[member_df.AgeAtFirstClaim == '60-69', 'AgeAtFirstClaim'] = 65
member_df.loc[member_df.AgeAtFirstClaim == '70-79', 'AgeAtFirstClaim'] = 75
member_df.loc[member_df.AgeAtFirstClaim == '80+', 'AgeAtFirstClaim'] = 80

#replace missing values
member_df.loc[member_df.AgeAtFirstClaim.isna(), 'AgeAtFirstClaim'] = 45 

# Claims process

In [6]:
claims_df = pd.read_csv("3_Claims.csv")
#claims_df.info()
claims_df.columns = ['MemberID', 'Provider', 'Vendor', 'PCP', 'Year', 'Specialty',
       'Place', 'PayDelay', 'LOS', 'DSFS', 'PCG',
       'Charlson', 'Procedure', 'SupLOS']

In [7]:
claims_df['Full_count'] = claims_df.apply(lambda x: x.count(), axis=1)
claims_df

,MemberID,Provider,Vendor,PCP,Year,Specialty,Place,PayDelay,LOS,DSFS,PCG,Charlson,Procedure,SupLOS,Full_count
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,NaN,8- 9 months,NEUMENT,0,MED,0,13
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,NaN,7- 8 months,NEUMENT,1-2,EM,0,13
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,NaN,0- 1 month,METAB3,0,EM,0,13
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,NaN,5- 6 months,METAB3,1-2,SCS,0,13
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,NaN,4- 5 months,FXDISLC,1-2,EM,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2668985,14932948,6641119.0,693028.0,22193.0,Y2,Obstetrics and Gynecology,Inpatient Hospital,58,NaN,0- 1 month,GYNEC1,0,EM,0,13
2668986,31248189,6932712.0,223304.0,70748.0,Y3,Internal,Inpatient Hospital,23,NaN,0- 1 month,GIBLEED,1-2,EM,0,13
2668987,43767339,1483429.0,35565.0,5278.0,Y3,Diagnostic Imaging,Office,122,NaN,4- 5 months,ODaBNCA,0,SIS,0,13
2668988,96393713,7094351.0,347045.0,93075.0,Y3,Internal,Office,151,NaN,1- 2 months,METAB3,1-2,EM,0,13


In [8]:
claims_df = claims_df[claims_df.Full_count > 6]
claims_df = claims_df.query("SupLOS != 1")
claims_df = claims_df.drop(['Full_count'], axis = 1)
claims_df

,MemberID,Provider,Vendor,PCP,Year,Specialty,Place,PayDelay,LOS,DSFS,PCG,Charlson,Procedure,SupLOS
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,NaN,8- 9 months,NEUMENT,0,MED,0
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,NaN,7- 8 months,NEUMENT,1-2,EM,0
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,NaN,0- 1 month,METAB3,0,EM,0
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,NaN,5- 6 months,METAB3,1-2,SCS,0
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,NaN,4- 5 months,FXDISLC,1-2,EM,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2668985,14932948,6641119.0,693028.0,22193.0,Y2,Obstetrics and Gynecology,Inpatient Hospital,58,NaN,0- 1 month,GYNEC1,0,EM,0
2668986,31248189,6932712.0,223304.0,70748.0,Y3,Internal,Inpatient Hospital,23,NaN,0- 1 month,GIBLEED,1-2,EM,0
2668987,43767339,1483429.0,35565.0,5278.0,Y3,Diagnostic Imaging,Office,122,NaN,4- 5 months,ODaBNCA,0,SIS,0
2668988,96393713,7094351.0,347045.0,93075.0,Y3,Internal,Office,151,NaN,1- 2 months,METAB3,1-2,EM,0


### PayDelay

In [10]:
claims_df.loc[claims_df['PayDelay']=='162+','PayDelay'] = 162
claims_df['PayDelay'] = claims_df['PayDelay'].astype(int)
claims_df = claims_df.query("PayDelay != 162")

### Length of Stay

In [11]:
claims_df.loc[claims_df['LOS']=='1 day','LOS'] = 1
claims_df.loc[claims_df['LOS']=='2 days','LOS'] = 2
claims_df.loc[claims_df['LOS']=='3 days','LOS'] = 3
claims_df.loc[claims_df['LOS']=='4 days','LOS'] = 4
claims_df.loc[claims_df['LOS']=='5 days','LOS'] = 5
claims_df.loc[claims_df['LOS']=='6 days','LOS'] = 6
claims_df.loc[claims_df['LOS']=='1- 2 weeks','LOS'] = 11
claims_df.loc[claims_df['LOS']=='2- 4 weeks','LOS'] = 21
claims_df.loc[claims_df['LOS']=='4- 8 weeks','LOS'] = 42
claims_df.loc[claims_df['LOS']=='26+ weeks','LOS'] = 180

claims_df['LOS'].fillna(0, inplace=True)
claims_df['LOS'] = claims_df['LOS'].astype(int)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [18]:
claims_df

,MemberID,Provider,Vendor,PCP,Year,Specialty,Place,PayDelay,LOS,DSFS,PCG,Charlson,Procedure,SupLOS
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,0,8- 9 months,NEUMENT,0,MED,0
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,0,7- 8 months,NEUMENT,1-2,EM,0
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,0,0- 1 month,METAB3,0,EM,0
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,0,5- 6 months,METAB3,1-2,SCS,0
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,0,4- 5 months,FXDISLC,1-2,EM,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2668985,14932948,6641119.0,693028.0,22193.0,Y2,Obstetrics and Gynecology,Inpatient Hospital,58,0,0- 1 month,GYNEC1,0,EM,0
2668986,31248189,6932712.0,223304.0,70748.0,Y3,Internal,Inpatient Hospital,23,0,0- 1 month,GIBLEED,1-2,EM,0
2668987,43767339,1483429.0,35565.0,5278.0,Y3,Diagnostic Imaging,Office,122,0,4- 5 months,ODaBNCA,0,SIS,0
2668988,96393713,7094351.0,347045.0,93075.0,Y3,Internal,Office,151,0,1- 2 months,METAB3,1-2,EM,0


## Separate the data for 3 years

In [19]:
claims_Y1_df = claims_df.query("Year == 'Y1'").drop(['Year'], axis=1).reset_index(drop=True) 
claims_Y2_df = claims_df.query("Year == 'Y2'").drop(['Year'], axis=1).reset_index(drop=True) 
claims_Y3_df = claims_df.query("Year == 'Y3'").drop(['Year'], axis=1).reset_index(drop=True)

##  Process data of Unique MemberID

In [20]:
#compute the number of provider, vendor, claim, ... for each unique member
claims_Y1_grouped = pd.DataFrame(claims_Y1_df.groupby(['MemberID']).agg({
    'Provider': ['count','nunique'],
    'Specialty' : 'nunique',
    'PCG' : 'nunique',
    'Procedure' : 'nunique',
    'PayDelay' : 'sum',
    'LOS' : 'sum'
}).reset_index())
claims_Y1_grouped.columns = ['MemberID', 'no_Claims', 'no_Providers','no_Specialties','no_PCG',\
                             'no_Procedure','sum_PayDelay','sum_LOS']

#one-hot encoding
claims_Y1_cat = claims_Y1_df.select_dtypes(include=['object'])
cat_cols = list(claims_Y1_cat.columns)

for col in cat_cols:
  onehot_features = pd.get_dummies(claims_Y1_cat[col])
  labels = list(onehot_features.columns) 
  for i in range(len(labels)):
    col_encode = col + "_" + labels[i]
    claims_Y1_df[col_encode] = onehot_features[onehot_features.columns[i]]
  claims_Y1_df = claims_Y1_df.drop([col], axis=1)


claims_Y1_encode = claims_Y1_df.reset_index(drop=True)
claims_Y1_encode = claims_Y1_encode.drop(['Provider','Vendor','PCP','LOS','SupLOS','PayDelay'],axis =1)
claims_Y1_encode.sort_values('MemberID',ignore_index=True)

#count one-hot values base on memberID
Y1_encode_group = pd.DataFrame()
Y1_encode_group['MemberID'] = claims_Y1_grouped['MemberID']
count_unique = list(claims_Y1_encode.columns)
count_unique.remove('MemberID')
for col in count_unique:
  df = claims_Y1_encode.groupby(['MemberID'])[col].agg('sum')
  Y1_encode_group = pd.merge(Y1_encode_group, df, on=['MemberID'])


claims_Y1_processed = pd.merge(claims_Y1_grouped, Y1_encode_group, on=['MemberID'])
claims_Y1_processed

,MemberID,no_Claims,no_Providers,no_Specialties,no_PCG,no_Procedure,sum_PayDelay,sum_LOS,Specialty_Anesthesiology,Specialty_Diagnostic Imaging,Specialty_Emergency,Specialty_General Practice,Specialty_Internal,Specialty_Laboratory,Specialty_Obstetrics and Gynecology,Specialty_Other,Specialty_Pathology,Specialty_Pediatrics,Specialty_Rehabilitation,Specialty_Surgery,Place_Ambulance,Place_Home,Place_Independent Lab,Place_Inpatient Hospital,Place_Office,Place_Other,Place_Outpatient Hospital,Place_Urgent Care,DSFS_0- 1 month,DSFS_1- 2 months,DSFS_10-11 months,DSFS_11-12 months,DSFS_2- 3 months,DSFS_3- 4 months,DSFS_4- 5 months,DSFS_5- 6 months,DSFS_6- 7 months,DSFS_7- 8 months,DSFS_8- 9 months,DSFS_9-10 months,...,PCG_MSC2a3,PCG_NEUMENT,PCG_ODaBNCA,PCG_PERINTL,PCG_PERVALV,PCG_PNCRDZ,PCG_PNEUM,PCG_PRGNCY,PCG_RENAL1,PCG_RENAL2,PCG_RENAL3,PCG_RESPR4,PCG_ROAMI,PCG_SEIZURE,PCG_SEPSIS,PCG_SKNAUT,PCG_STROKE,PCG_TRAUMA,PCG_UTI,Charlson_0,Charlson_1-2,Charlson_3-4,Charlson_5+,Procedure_ANES,Procedure_EM,Procedure_MED,Procedure_PL,Procedure_RAD,Procedure_SAS,Procedure_SCS,Procedure_SDS,Procedure_SEOA,Procedure_SGS,Procedure_SIS,Procedure_SMCD,Procedure_SMS,Procedure_SNS,Procedure_SO,Procedure_SRS,Procedure_SUS
0,210,6,3,3,3,4,396,2,0,0,1,0,3,2,0,0,0,0,0,0,0,0,2,0,3,0,0,1,2,2,0,0,0,0,0,0,1,0,0,1,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,3,1,0,0,1,1,0,0,0,0,0,0,0,0,0
1,3197,2,2,2,1,1,132,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3889,12,7,4,5,5,757,3,0,1,4,0,3,4,0,0,0,0,0,0,0,0,4,0,3,0,0,5,11,1,0,0,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,2,0,0,8,4,0,0,0,4,0,4,1,0,1,2,0,0,0,0,0,0,0,0,0
3,4187,3,3,3,3,2,178,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,9063,4,2,2,1,2,241,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75127,99995554,33,3,3,3,4,3575,0,0,0,0,2,24,7,0,0,0,0,0,0,0,0,7,0,26,0,0,0,1,0,0,0,2,0,1,4,7,7,4,7,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,0,0,0,33,0,0,0,0,4,22,3,0,0,4,0,0,0,0,0,0,0,0,0,0
75128,99996214,1,1,1,1,1,19,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
75129,99997485,1,1,1,1,1,130,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
75130,99997895,14,5,4,6,4,539,0,0,1,0,0,4,7,0,0,0,0,0,2,0,0,7,0,7,0,0,0,2,1,0,0,1,2,2,0,0,0,3,1,...,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,14,0,0,0,0,6,0,4,1,0,3,0,0,0,0,0,0,0,0,0,0


In [21]:
#compute the number of provider, vendor, claim, ... for each unique member
claims_Y2_grouped = pd.DataFrame(claims_Y2_df.groupby(['MemberID']).agg({
    'Provider': ['count','nunique'],
    'Specialty' : 'nunique',
    'PCG' : 'nunique',
    'Procedure' : 'nunique',
    'PayDelay' : 'sum',
    'LOS' : 'sum'
}).reset_index())
claims_Y2_grouped.columns = ['MemberID', 'no_Claims', 'no_Providers','no_Specialties','no_PCG',\
                             'no_Procedure','sum_PayDelay','sum_LOS']

#one-hot encoding
claims_Y2_cat = claims_Y2_df.select_dtypes(include=['object'])
cat_cols = list(claims_Y2_cat.columns)

for col in cat_cols:
  onehot_features = pd.get_dummies(claims_Y2_cat[col])
  labels = list(onehot_features.columns) 
  for i in range(len(labels)):
    col_encode = col + "_" + labels[i]
    claims_Y2_df[col_encode] = onehot_features[onehot_features.columns[i]]
  claims_Y2_df = claims_Y2_df.drop([col], axis=1)

claims_Y2_encode = claims_Y2_df.reset_index(drop=True)
claims_Y2_encode = claims_Y2_encode.drop(['Provider','Vendor','PCP','LOS','SupLOS','PayDelay'],axis =1)
claims_Y2_encode.sort_values('MemberID',ignore_index=True)

#count one-hot values base on memberID
Y2_encode_group = pd.DataFrame()
Y2_encode_group['MemberID'] = claims_Y2_grouped['MemberID']
count_unique = list(claims_Y2_encode.columns)
count_unique.remove('MemberID')
for col in count_unique:
  df = claims_Y2_encode.groupby(['MemberID'])[col].agg('sum')
  Y2_encode_group = pd.merge(Y2_encode_group, df, on=['MemberID'])

claims_Y2_processed = pd.merge(claims_Y2_grouped, Y2_encode_group, on=['MemberID'])
claims_Y2_processed

,MemberID,no_Claims,no_Providers,no_Specialties,no_PCG,no_Procedure,sum_PayDelay,sum_LOS,Specialty_Anesthesiology,Specialty_Diagnostic Imaging,Specialty_Emergency,Specialty_General Practice,Specialty_Internal,Specialty_Laboratory,Specialty_Obstetrics and Gynecology,Specialty_Other,Specialty_Pathology,Specialty_Pediatrics,Specialty_Rehabilitation,Specialty_Surgery,Place_Ambulance,Place_Home,Place_Independent Lab,Place_Inpatient Hospital,Place_Office,Place_Other,Place_Outpatient Hospital,Place_Urgent Care,DSFS_0- 1 month,DSFS_1- 2 months,DSFS_10-11 months,DSFS_11-12 months,DSFS_2- 3 months,DSFS_3- 4 months,DSFS_4- 5 months,DSFS_5- 6 months,DSFS_6- 7 months,DSFS_7- 8 months,DSFS_8- 9 months,DSFS_9-10 months,...,PCG_MSC2a3,PCG_NEUMENT,PCG_ODaBNCA,PCG_PERINTL,PCG_PERVALV,PCG_PNCRDZ,PCG_PNEUM,PCG_PRGNCY,PCG_RENAL1,PCG_RENAL2,PCG_RENAL3,PCG_RESPR4,PCG_ROAMI,PCG_SEIZURE,PCG_SEPSIS,PCG_SKNAUT,PCG_STROKE,PCG_TRAUMA,PCG_UTI,Charlson_0,Charlson_1-2,Charlson_3-4,Charlson_5+,Procedure_ANES,Procedure_EM,Procedure_MED,Procedure_PL,Procedure_RAD,Procedure_SAS,Procedure_SCS,Procedure_SDS,Procedure_SEOA,Procedure_SGS,Procedure_SIS,Procedure_SMCD,Procedure_SMS,Procedure_SNS,Procedure_SO,Procedure_SRS,Procedure_SUS
0,4,1,1,1,1,1,43,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,210,6,3,3,2,3,308,0,0,0,0,0,1,1,0,4,0,0,0,0,0,0,1,0,5,0,0,0,2,0,0,0,0,2,0,0,1,0,0,1,...,1,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3197,5,4,4,2,4,148,0,0,1,0,0,1,2,0,0,0,1,0,0,0,0,2,0,3,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,5,0,0,0,0,2,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0
3,3457,1,1,1,1,1,63,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3713,10,5,3,5,3,562,0,0,1,0,7,0,2,0,0,0,0,0,0,0,0,2,0,8,0,0,0,1,5,0,0,0,0,0,0,0,0,4,0,...,2,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,2,10,0,0,0,0,5,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71066,99985996,4,1,1,2,2,154,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,2,1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71067,99987030,4,3,2,1,3,125,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
71068,99995391,11,2,2,4,4,533,0,0,0,0,0,6,0,0,0,0,0,0,5,0,0,0,0,11,0,0,0,2,1,0,0,4,2,0,2,0,0,0,0,...,0,5,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,11,0,0,0,0,3,6,1,1,0,0,0,0,0,0,0,0,0,0,0,0
71069,99995554,15,3,3,3,4,581,0,0,0,0,2,11,2,0,0,0,0,0,0,0,0,2,0,13,0,0,0,10,2,0,0,1,2,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,11,0,0,0,15,0,0,0,0,2,11,1,0,0,1,0,0,0,0,0,0,0,0,0,0


# Drug Count & Lab Count

In [22]:
drug_df = pd.read_csv("DrugCount.csv")
# Replace DrugCount 7+ with 7
drug_df.loc[drug_df['DrugCount']=='7+','DrugCount'] = 7
drug_df['DrugCount'] = drug_df['DrugCount'].astype(int)
drug_df = drug_df.query("DrugCount != 7")
drug_df = drug_df.drop(['DSFS'], axis = 1)

lab_df = pd.read_csv("LabCount.csv")
# Replace LabCount 10+ with 10
lab_df.loc[lab_df['LabCount']=='10+', 'LabCount'] = 10
lab_df['LabCount'] = lab_df['LabCount'].astype(int)
lab_df = lab_df.query("LabCount != 10")
lab_df = lab_df.drop(['DSFS'], axis = 1)

In [23]:
drug_Y1_df = drug_df.query("Year == 'Y1'").drop(['Year'], axis = 1).reset_index(drop=True) 
drug_Y2_df = drug_df.query("Year == 'Y2'").drop(['Year'], axis = 1).reset_index(drop=True) 

lab_Y1_df = lab_df.query("Year == 'Y1'").drop(['Year'], axis = 1).reset_index(drop=True) 
lab_Y2_df = lab_df.query("Year == 'Y2'").drop(['Year'], axis = 1).reset_index(drop=True) 

In [40]:
drug_Y1_df.sort_values('MemberID', ignore_index=True)
drug_Y1_group = pd.DataFrame(drug_Y1_df.groupby(['MemberID'])['DrugCount'].agg('sum').reset_index())
drug_Y1_group.columns = ['MemberID','DrugSum']

drug_Y1 = pd.merge(claims_Y1_grouped['MemberID'], drug_Y1_group, on = 'MemberID', how = 'left')
drug_Y1.fillna(0, inplace=True)
drug_Y1['DrugSum'] = drug_Y1['DrugSum'].astype(int)
drug_Y1

,MemberID,DrugSum
0,210,5
1,3197,5
2,3889,30
3,4187,19
4,9063,2
...,...,...
75127,99995554,8
75128,99996214,0
75129,99997485,0
75130,99997895,0


In [39]:
drug_Y2_df.sort_values('MemberID', ignore_index=True)
drug_Y2_group = pd.DataFrame(drug_Y1_df.groupby(['MemberID'])['DrugCount'].agg('sum').reset_index())
drug_Y2_group.columns = ['MemberID','DrugSum']

drug_Y2 = pd.merge(claims_Y2_grouped['MemberID'], drug_Y2_group, on = 'MemberID', how = 'left')
drug_Y2.fillna(0, inplace=True)
drug_Y2['DrugSum'] = drug_Y2['DrugSum'].astype(int)
drug_Y2

,MemberID,DrugSum
0,4,0
1,210,5
2,3197,5
3,3457,0
4,3713,0
...,...,...
71066,99985996,0
71067,99987030,0
71068,99995391,0
71069,99995554,8


In [41]:
lab_Y1_df.sort_values('MemberID', ignore_index=True)
lab_Y1_group = pd.DataFrame(lab_Y1_df.groupby(['MemberID'])['LabCount'].agg('sum').reset_index())
lab_Y1_group.columns = ['MemberID','LabSum']

lab_Y1 = pd.merge(claims_Y1_grouped['MemberID'], lab_Y1_group, on = 'MemberID', how = 'left')
lab_Y1.fillna(0, inplace=True)
lab_Y1['LabSum'] = lab_Y1['LabSum'].astype(int)
lab_Y1

,MemberID,LabSum
0,210,2
1,3197,0
2,3889,0
3,4187,0
4,9063,0
...,...,...
75127,99995554,11
75128,99996214,0
75129,99997485,0
75130,99997895,10


In [42]:
lab_Y2_df.sort_values('MemberID', ignore_index=True)
lab_Y2_group = pd.DataFrame(lab_Y2_df.groupby(['MemberID'])['LabCount'].agg('sum').reset_index())
lab_Y2_group.columns = ['MemberID','LabSum']

lab_Y2 = pd.merge(claims_Y2_grouped['MemberID'], lab_Y2_group, on = 'MemberID', how = 'left')
lab_Y2.fillna(0, inplace=True)
lab_Y2['LabSum'] = lab_Y2['LabSum'].astype(int)
lab_Y2

,MemberID,LabSum
0,4,0
1,210,1
2,3197,2
3,3457,0
4,3713,9
...,...,...
71066,99985996,0
71067,99987030,0
71068,99995391,1
71069,99995554,0


In [43]:
labdrug_Y1 = pd.merge(drug_Y1, lab_Y1, on=['MemberID'], how='outer')
labdrug_Y1 = labdrug_Y1.sort_values('MemberID', ignore_index=True)

labdrug_Y2 = pd.merge(drug_Y2, lab_Y2, on=['MemberID'], how='outer')
labdrug_Y2 = labdrug_Y2.sort_values('MemberID', ignore_index=True)

# Targets Observation

In [46]:
targetY1_DIHY2_df = pd.read_csv("DaysInHospital_Y3.csv")
targetY1_DIHY2_df['CLASS'] = -1
targetY1_DIHY2_df.loc[targetY1_DIHY2_df.DaysInHospital != 0, 'CLASS'] = 1
targetY1_DIHY2_df = targetY1_DIHY2_df.query("ClaimsTruncated != 1").drop(['ClaimsTruncated'], axis = 1).reset_index(drop=True)
targetY1_DIHY2_df

,MemberID,DaysInHospital,CLASS
0,90963501,0,-1
1,85160905,0,-1
2,45496648,0,-1
3,45162557,0,-1
4,68793937,0,-1
...,...,...,...
67386,206785,0,-1
67387,93465383,0,-1
67388,57466586,0,-1
67389,57283229,0,-1


In [47]:
targetY2_DIHY3_df = pd.read_csv("DaysInHospital_Y3.csv")
targetY2_DIHY3_df['CLASS'] = -1
targetY2_DIHY3_df.loc[targetY2_DIHY3_df.DaysInHospital != 0, 'CLASS'] = 1
targetY2_DIHY3_df = targetY2_DIHY3_df.query("ClaimsTruncated != 1").drop(['ClaimsTruncated'], axis = 1).reset_index(drop=True)
targetY2_DIHY3_df

,MemberID,DaysInHospital,CLASS
0,90963501,0,-1
1,85160905,0,-1
2,45496648,0,-1
3,45162557,0,-1
4,68793937,0,-1
...,...,...,...
67386,206785,0,-1
67387,93465383,0,-1
67388,57466586,0,-1
67389,57283229,0,-1


In [48]:
targetY2 = pd.DataFrame()

for id in claims_Y2_processed.MemberID:
  targetY2 = targetY2.append(targetY2_DIHY3_df.loc[targetY2_DIHY3_df.MemberID == id])
targetY2.sort_values('MemberID',ignore_index=True)

,MemberID,DaysInHospital,CLASS
0,4,0,-1
1,210,0,-1
2,3197,0,-1
3,3457,0,-1
4,3713,0,-1
...,...,...,...
67022,99985996,0,-1
67023,99987030,0,-1
67024,99995391,0,-1
67025,99995554,0,-1


# Save the processed data

In [56]:
%cd /content/drive/My Drive/HHP_release3/data_processed/

/content/drive/My Drive/HHP_release3/data_processed


In [57]:
Y1_data = pd.merge(claims_Y1_processed, labdrug_Y1, on=['MemberID'])
Y2_data = pd.merge(claims_Y2_processed, labdrug_Y2, on=['MemberID'])

In [58]:
Y1_fulldata = pd.merge(Y1_data, targetY1_DIHY2_df, on=['MemberID'])
Y1_fulldata.rename(columns={"DaysInHospital": "TARGET"}, inplace=True)
Y1_fulldata.to_csv('dataY1.csv', index=False)

In [59]:
Y2_fulldata = pd.merge(Y2_data, targetY2, on=['MemberID'])
Y2_fulldata.rename(columns={"DaysInHospital": "TARGET"}, inplace=True)
Y2_fulldata.to_csv('dataY2.csv', index=False)

In [60]:
member_df.sort_values('MemberID').to_csv('Member_processed.csv', index=False)

In [61]:
memberY1_df = pd.DataFrame()
for id in Y1_data.MemberID:
  memberY1_df = memberY1_df.append(member_df.loc[member_df.MemberID == id])
memberY1_df.sort_values('MemberID',ignore_index=True)

memberY2_df = pd.DataFrame()
for id in Y2_data.MemberID:
  memberY2_df = memberY2_df.append(member_df.loc[member_df.MemberID == id])
memberY2_df.sort_values('MemberID',ignore_index=True)

,MemberID,AgeAtFirstClaim,Male,Female,Unknown
0,4,5,1,0,0
1,210,35,0,0,1
2,3197,5,0,1,0
3,3457,5,1,0,0
4,3713,45,0,1,0
...,...,...,...,...,...
71066,99985996,45,0,1,0
71067,99987030,25,1,0,0
71068,99995391,65,0,0,1
71069,99995554,45,1,0,0


In [62]:
Y1_df = pd.merge(memberY1_df, Y1_fulldata, on=['MemberID']).to_csv('fulldataY1.csv', index=False)
Y2_df = pd.merge(memberY2_df, Y2_fulldata, on=['MemberID']).to_csv('fulldataY2.csv', index=False)

In [64]:
memberY1_df.sort_values('MemberID').to_csv('MBlistY1.csv', index=False)
memberY2_df.sort_values('MemberID').to_csv('MBlistY2.csv', index=False)

In [65]:
claims_Y1_processed.sort_values('MemberID').to_csv('ClaimsY1_data.csv', index=False)
claims_Y2_processed.sort_values('MemberID').to_csv('ClaimsY2_data.csv', index=False)

In [66]:
labdrug_Y1.sort_values('MemberID').to_csv('LabDrugY1.csv', index=False)
labdrug_Y2.sort_values('MemberID').to_csv('LabDrugY2.csv', index=False)

In [67]:
targetY1_DIHY2_df.sort_values('MemberID').to_csv('DaysInHos_Y1.csv', index=False)
targetY2.sort_values('MemberID').to_csv('DaysInHos_Y2.csv', index=False)